In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

In [0]:
emp_schema = ['id','name','salary','department','gender']

In [0]:
emp_df = spark.createDataFrame(data=emp_data,schema = emp_schema)

In [0]:
from pyspark.sql.functions import *

In [0]:
#Groupby just gives distinct records not all the records grouped, so it is a limitation 
emp_df.groupBy("department")\
  .agg(sum("salary")).show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|        IT|     345000|
| marketing|     220000|
|     sales|     240000|
+----------+-----------+



In [0]:
#So using window function we get all the values grouped in their particular windows
from pyspark.sql.window import Window

window = Window.partitionBy("department")
emp_df.withColumn("total_sal",sum(col("salary")).over(window)) \
  .show(truncate=False)
  

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-4467348028316252>:5
      2 from pyspark.sql.window import Window
      4 window = Window.partitionBy("department")
----> 5 emp_df.withColumn("total_sal",sum(col("salary")).over(window)) \
      6   .show(truncate=False)

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [0]:
#Using row_number, Rank and Desnse_Rank in the window function
window = Window.partitionBy("department").orderBy("salary")

emp_df = emp_df.withColumn("row_number", row_number().over(window)) \
               .withColumn("Rank", rank().over(window)) \
               .withColumn("Dense_Rank", dense_rank().over(window))\
                 .show(truncate=False)

+---+--------+------+----------+------+----------+----+----------+
|id |name    |salary|department|gender|row_number|Rank|Dense_Rank|
+---+--------+------+----------+------+----------+----+----------+
|1  |manish  |50000 |IT        |m     |1         |1   |1         |
|11 |rakhi   |50000 |IT        |f     |2         |1   |1         |
|9  |aditya  |65000 |IT        |m     |3         |3   |2         |
|4  |mukesh  |80000 |IT        |m     |4         |4   |3         |
|8  |rashi   |100000|IT        |f     |5         |5   |4         |
|6  |nikita  |45000 |marketing |f     |1         |1   |1         |
|10 |rahul   |50000 |marketing |m     |2         |2   |2         |
|7  |ragini  |55000 |marketing |f     |3         |3   |3         |
|3  |raushan |70000 |marketing |m     |4         |4   |4         |
|2  |vikash  |60000 |sales     |m     |1         |1   |1         |
|5  |priti   |90000 |sales     |f     |2         |2   |2         |
|12 |akhilesh|90000 |sales     |m     |3         |2   |2      